In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
import prometheux_chain as pmtx

In [2]:
import os

PMTX_TOKEN = "dev-token"
os.environ["PMTX_TOKEN"] = PMTX_TOKEN

#### Project functions

In [3]:
pmtx.cleanup_projects()

All user's projects deleted successfully


In [4]:
project_id = pmtx.save_project(project_name="test_project")
project_id

'203d8aec650'

In [5]:
pmtx.list_projects()

[{'id': '203d8aec650',
  'name': 'test_project',
  'author': 'devuser',
  'scope': 'user',
  'to_persist': True,
  'created_at': '2025-08-14T16:44:37.181247+00:00',
  'updated_at': '2025-08-14T16:44:37.181247+00:00'}]

In [6]:
project = pmtx.load_project(project_id=project_id)
project

{'id': '203d8aec650',
 'name': 'test_project',
 'author': 'devuser',
 'scope': 'user',
 'to_persist': True,
 'created_at': '2025-08-14T16:44:37.181247+00:00',
 'updated_at': '2025-08-14T16:44:37.181247+00:00'}

#### Data functions

In [7]:
pmtx.cleanup_sources()

All data sources for workspace workspace_id deleted successfully


In [8]:
db = pmtx.data.database.Database(
    database_type="csv",
    username="",
    password="",
    host=os.getcwd() + "/../resources/ownership",
    port=0000,
    database_name="ownership.csv",
    options={} #{"delimiter": "\t"}
)
pmtx.connect_sources(database_payload=db)

{'connectionStatus': True,
 'sources': [{'id': '8daab3d610af52f0b3c2b30710311724',
   'bind_annotation': '@bind("ownership_csv","csv useHeaders=\'true\'","/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/notebooks/../resources/ownership","ownership.csv").',
   'model_annotation': '@model("ownership_csv", "Owner:string,Company:string,Share:double", "").',
   'rule_block': '',
   'row_count': 0,
   'column_count': 3,
   'author': 'devuser',
   'timestamp': '2025-08-14T16:44:37.560456+00:00'}],
 'errorMessage': None}

In [9]:
pmtx.list_sources()

[{'id': '8daab3d610af52f0b3c2b30710311724',
  'bind_annotation': '@bind("ownership_csv","csv useHeaders=\'true\'","/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/notebooks/../resources/ownership","ownership.csv").',
  'model_annotation': '@model("ownership_csv", "Owner:string,Company:string,Share:double", "").',
  'rule_block': '',
  'row_count': 0,
  'column_count': 3,
  'author': 'devuser',
  'timestamp': '2025-08-14T16:44:37.560456+00:00'}]

#### Concept functions

In [10]:
pmtx.cleanup_concepts(project_id=project_id)

In [11]:
concept_logic = """
ownership(Owner,Company,Shares) :- ownership_csv(Owner,Company,Shares).
@output("ownership").
"""

pmtx.save_concept(project_id=project_id, concept_logic=concept_logic)

In [12]:
concept_logic = """
owner(Owner) :- ownership(Owner,Company,Shares).
@output("owner").
"""

pmtx.save_concept(project_id=project_id, concept_logic=concept_logic)

In [13]:
pmtx.run_concept(project_id=project_id, concept_name="ownership", persist_outputs=True)

{'message': "Successfully ran concept 'ownership'",
 'concept_name': 'ownership',
 'predicates_populated': [],
 'total_records': 0,
 'evaluation_results': {'id': 33,
  'resultSet': {},
  'types': {'ownership': ['STRING', 'STRING', 'DOUBLE']},
  'columnNames': {'ownership': ['Owner', 'Company', 'Shares']}}}

In [14]:
pmtx.list_concepts(project_id=project_id)

[{'predicate_name': 'owner',
  'fields': {'Owner': 'unknown'},
  'column_count': 1,
  'is_input': False,
  'row_count': 0,
  'type': 'api',
  'timestamp': '2025-08-14T16:44:37.850103+00:00',
  'author': 'devuser'},
 {'predicate_name': 'ownership',
  'fields': {'Owner': 'string', 'Company': 'string', 'Shares': 'double'},
  'column_count': 3,
  'is_input': False,
  'row_count': 0,
  'type': 'app',
  'timestamp': '2025-08-14T16:44:38.364037+00:00',
  'author': 'devuser'},
 {'predicate_name': 'ownership_csv',
  'fields': {'Owner': 'string', 'Company': 'string', 'Share': 'double'},
  'column_count': 3,
  'is_input': True,
  'row_count': 0,
  'type': 'csv',
  'timestamp': '2025-08-14T16:44:37.560456+00:00',
  'author': 'devuser'}]

#### GraphRAG function

In [15]:
graph_rag_result = pmtx.graph_rag(project_id=project_id,
                                  question="This is the question", 
                                  rag_concepts=[{"concept": "ownership", "field_to_embed": "Owner"}],
                                  graph_concepts=["owner"])

graph_rag_result['graph_rag_results']

{'owner(Owner)': [['Acme Inc'],
  ['John Doe'],
  ['Umbrella Corp'],
  ['Daniel Black']]}